# ¿POR QUÉ Y CUANDO DEBO ESCALAR LOS DATOS?

En este notebook vamos a analizar el efecto que el escalamiento de los datos tiene en el desempeño de diferentes modelos de *Machine Learning*.

Y con los resultados que obtengamos tendremos claro cuando resulta conveniente escalar los datos y cuando no.

## 1. El problema a resolver

Usaremos un set de datos que contiene la información asociada al pago ("no default") e impago ("default") de un crédito bancario para un total de 5.383 personas.

Construiremos varios modelos de *Machine Learning* que tomarán como entrada datos como el puntaje crediticio, los ingresos mensuales y el monto del préstamo, y aprenda a predecir si el usuario pagará a tiempo ("no default", 0) o se retrasará en sus pagos ("default", 1).

Y cada modelo será entrenado con datos con y sin escalar y veremos el efecto que esto tiene en cada modelo.

In [1]:
# Importar las librerías
import pandas as pd
import plotly.express as px
from google.colab import drive

ModuleNotFoundError: No module named 'plotly'

## 2. El set de datos

In [ ]:
# Leer el dataset
RUTA = '/content/dataset_default.csv'
df = pd.read_csv(RUTA)
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45184,51,technician,married,tertiary,no,825.0,no,no,cellular,17,nov,977.0,3,-1.0,0,unknown,yes
45185,71,retired,divorced,primary,no,1729.0,no,no,cellular,17,nov,456.0,2,-1.0,0,unknown,yes
45186,72,retired,married,secondary,no,5715.0,no,no,cellular,17,nov,1127.0,5,184.0,3,success,yes
45187,57,blue-collar,married,secondary,no,668.0,no,no,telephone,17,nov,508.0,4,-1.0,0,unknown,no


Veamos un conteo de categorías para verificar si el set de datos está balanceado:

In [ ]:
df['job'].value_counts(normalize=True)

blue-collar       0.215251
management        0.209232
technician        0.168005
administrative    0.114364
services          0.091881
retired           0.050079
self-employed     0.034920
entrepreneur      0.032884
unemployed        0.028834
housemaid         0.027440
student           0.020735
unknown           0.006373
Name: job, dtype: float64

Veamos los rangos de valores que alcanzan los datos que usaremos como entrada de cada modelo:

In [ ]:
for col in df.columns[0:3]:
    # Convertir los valores de la columna a números de punto flotante
    numeric_values = pd.to_numeric(df[col], errors='coerce')

    # Filtrar los valores nulos si es necesario
    numeric_values = numeric_values.dropna()

    # Calcular valores mínimo y máximo de la columna
    if not numeric_values.empty:
        minimo = numeric_values.min()
        maximo = numeric_values.max()
        print(f'Máximo/mínimo columna "{col}": {maximo:.1f}/{minimo:.1f}')
    else:
        print(f'No se pueden calcular el máximo/mínimo de la columna "{col}" debido a datos no numéricos')


Máximo/mínimo columna "age": 95.0/18.0
No se pueden calcular el máximo/mínimo de la columna "job" debido a datos no numéricos
No se pueden calcular el máximo/mínimo de la columna "marital" debido a datos no numéricos


Y vemos que **cada columna tiene una escala diferente**.

Y por último hagamos un gráfico 3D de los datos para entender sus principales características:

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x='job', y='marital', z='loan', color='job',
                    symbol='loan', opacity=0.7, size_max=10,
                    title='Distribución de Trabajos según Estado Civil y Préstamo')
fig.update_layout(scene=dict(xaxis_title='Trabajo', yaxis_title='Estado Civil', zaxis_title='Préstamo'))
fig.show()

##3. ¿Qué es el escalamiento de los datos?

> El escalamiento consiste en hacer que cada variable(columna) tenga aproximadamente **el mismo rango de valores**.

Por ejemplo, si antes del escalamiento tenemos:

- `Puntaje crediticio`: [0.0 - 7.5]
- `Ingresos mensuales`: [1.4 - 6997.8]
- `Monto del préstamo`: [20023.4 - 49991.1]

Después del escalamiento podríamos tener:

- Todas las variables entre 0 y 1 (normalización)
- Todas las variables con valores promedio de 0 y desviaciones estándar de 1 (estandarización)

## 4. Funciones requeridas

Para introducir los datos a cada modelo necesitaremos:

1. obtener las entradas (`X`) y salida (`Y`) del modelo a partir del set de datos original
2. crear los sets de entrenamiento, validación y prueba a partir de `X` y `Y`

Además, para entrenar y validar cada modelo con los datos escalados necesitaremos crear una función para ESCALAR los datos.


In [ ]:
# Obtén los vectores X(características) e Y(target)
## A COMPLETAR

X = df.drop(columns=["loan"])  # Eliminamos la columna "loan" que no tiene sentido analizar
Y = df["job"]  # Cambiamos el nombre de la columna de objetivo por "job"


In [ ]:
from sklearn.model_selection import train_test_split

# Crear los sets de entrenamiento (70%), validación (15%) y prueba (15%)
x_train, x_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Verificar
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)


(31632, 16) (31632,)
(6778, 16) (6778,)
(6779, 16) (6779,)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

def escalar_datos(xtr, xvl, xts):
    '''Escalar datos de entrada (entrenamiento, validación y prueba)
    en el rango de 0 a 1'''
    scaler = MinMaxScaler()

    # Seleccionar solo las columnas numéricas
    num_cols = xtr.select_dtypes(include=['float64', 'int64']).columns

    # Fit + transform sobre el set de entrenamiento
    xtr_s = scaler.fit_transform(xtr[num_cols])

    # Transform sobre prueba y validación
    xvl_s = scaler.transform(xvl[num_cols])
    xts_s = scaler.transform(xts[num_cols])

    return xtr_s, xvl_s, xts_s

# Ejecutar la función
x_train_s, x_val_s, x_test_s = escalar_datos(x_train, x_val, x_test)

# Verificar
print(x_train_s.min(axis=0), x_train_s.max(axis=0))
print(x_val_s.min(axis=0), x_val_s.max(axis=0))
print(x_test_s.min(axis=0), x_test_s.max(axis=0))


[0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1.]
[0.         0.00739799 0.         0.00025773 0.         0.
 0.        ] [0.85714286 0.1395572  1.         1.26726804 0.94736842 0.96215596
 1.1372549 ]
[0.         0.01101109 0.         0.00025773 0.         0.
 0.        ] [0.93506494 0.20566855 1.         0.97525773 1.0877193  0.95412844
 0.78431373]


Ya tenemos las funciones básicas requeridas. Ahora sí veamos el efecto que tiene el escalamiento en diferentes tipos de modelos.

##5. Efecto del escalamiento en diferentes modelos de *Machine Learning*

En este experimento probaremos con dos tipos de modelos:

1. Un árbol de clasificación, que es "inmune" al escalamiento
2. Un clasificador kNN (*k-nearest neighbors*) que es sensible al escalamiento
3. Opcional (SVM)

Veamos el efecto que el escalamiento tiene sobre estos dos tipos de modelos:

###5.1. Árboles de clasificación

> En los árboles de clasificación (y los árboles de regresión y los bosques aleatorios) **el desempeño es el mismo con o sin escalamiento de los datos**.

Lo anterior se debe a que al momento de clasificar (o predecir un dato numérico) internamente los árboles/bosques calculan umbrales que están dados en la escala en la que se encuentren los datos:

![](https://drive.google.com/uc?export=view&id=1FhtonygNk7DfSoVooMQQGjoQGYQlXqJh
)

Como el umbral se desplaza en la misma proporción que lo hace el escalamiento, todos estos modelos tendrán el mismo desempeño.

Verifiquemos esto con nuestro set de datos.

Creemos una función para entrenar y evaluar un sencillo árbol de clasificación:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def entrenar_y_evaluar_arbol(X_train, y_train, X_test, y_test, max_depth=None):
    clf = DecisionTreeClassifier(max_depth=max_depth)
    clf.fit(X_train, y_train)

    # Predice el conjunto de entrenamiento y de prueba
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    # Calcula las precisiones del conjunto de entrenamiento y del de prueba
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    return train_accuracy, test_accuracy


Y ahora obtengamos el desempeño del árbol sin y con escalamiento:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Codificar las características categóricas usando one-hot encoding
X_encoded = pd.get_dummies(X)

# Divide los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.15, random_state=42)

# Entrena y evalúa el árbol sin escalamiento
train_accuracy_no_scaling, test_accuracy_no_scaling = entrenar_y_evaluar_arbol(X_train, y_train, X_test, y_test)

# Escala los datos y los entrena
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
train_accuracy_with_scaling, test_accuracy_with_scaling = entrenar_y_evaluar_arbol(X_train_scaled, y_train, X_test_scaled, y_test)

print("Rendimiento del árbol sin escalamiento:")
print("Precisión en entrenamiento:", train_accuracy_no_scaling)
print("Precisión en prueba:", test_accuracy_no_scaling)
print("---------------------------------------")
print("Rendimiento del árbol con escalamiento:")
print("Precisión en entrenamiento:", train_accuracy_with_scaling)
print("Precisión en prueba:", test_accuracy_with_scaling)


Rendimiento del árbol sin escalamiento:
Precisión en entrenamiento: 1.0
Precisión en prueba: 1.0
---------------------------------------
Rendimiento del árbol con escalamiento:
Precisión en entrenamiento: 1.0
Precisión en prueba: 1.0


###5.2. Clasificador kNN (*k-nearest neighbors*)

> En los clasificadores kNN **el desempeño NO necesariamente será el mismo con o sin escalamiento de los datos**.

Este clasificador toma el dato a clasificar y calcula su distancia con respecto a la totalidad de los datos. Luego determina la categoría con base en las categorías de los *k* datos más cercanos (con la menor distancia).

La clave es la forma como se calculan las distancias entre el punto a clasificar y los demás puntos con categorías conocidas:

![](https://drive.google.com/uc?export=view&id=1FkAVal4XEBb6HcpE4IS6yL5N7OEsCuaJ
)

Si cada característica tiene una escala diferente entonces aquellas características con mayores escalas tendrán un mayor efecto en el valor de la distancia obtenida.

Veamos nuevamente este comportamiento para el set de datos que estamos usando.

Creemos una función para entrenar y evaluar un clasificador kNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def train_and_evaluate_knn(X_train, y_train, X_test, y_test, n_neighbors=5):
    """
    Entrena y evalúa un clasificador kNN.

    Parámetros:
        - X_train: conjunto de características de entrenamiento.
        - y_train: etiquetas de clase correspondientes al conjunto de entrenamiento.
        - X_test: conjunto de características de prueba.
        - y_test: etiquetas de clase correspondientes al conjunto de prueba.
        - n_neighbors: número de vecinos a considerar (por defecto, 5).

    Retorna:
        - accuracy: precisión del clasificador en el conjunto de prueba.
    """
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Entrena el clasificador kNN
    knn.fit(X_train, y_train)

    # Predice las ETIQUETAS DE CLASE para el conjunto de prueba
    y_pred = knn.predict(X_test)

    # Calcula la precisión del clasificador
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy


In [ ]:
accuracy = train_and_evaluate_knn(X_train, y_train, X_test, y_test, n_neighbors=5)
print("Precisión del clasificador kNN:", accuracy)

Precisión del clasificador kNN: 0.20371736244283817


Y ahora calculemos el desempeño sin y con escalamiento:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Codifica las características categóricas usando ONE-HOT
X_encoded = pd.get_dummies(X)

# Divide los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.3, random_state=42)

# Entrena y evalúa el kNN SIN escalamiento
accuracy_without_scaling = train_and_evaluate_knn(X_train, Y_train, X_test, Y_test)
print("Precisión del clasificador kNN sin escalamiento:", accuracy_without_scaling)

# Escala los datos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar y evaluar el kNN CON escalamiento
accuracy_with_scaling = train_and_evaluate_knn(X_train_scaled, Y_train, X_test_scaled, Y_test)
print("Precisión del clasificador kNN con escalamiento:", accuracy_with_scaling)


Precisión del clasificador kNN sin escalamiento: 0.20166703547982592
Precisión del clasificador kNN con escalamiento: 0.9663642398760788


##6. Conclusiones

# Impacto del Escalamiento de Datos:
La precisión del clasificador kNN sin escalar es significativamente muy baja, con un valor de, aproximadamente, 0.20.

Después de aplicar el escalamiento, la precisión del kNN aumenta considerablemente: hasta el 0.97.

Este aumento en la precisión después del escalamiento sugiere que el rendimiento del kNN es **altamente sensible** a la escala de las características. Sin escalamiento, las características con magnitudes más grandes pueden dominar la contribución al cálculo de la distancia, lo que lleva a una clasificación incorrecta.

En este caso, el método de escalamiento que se utilizó la escala Min-Max para ajustar las características en un rango específico (0 a 1).